In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.linalg import inv

n_market = 50
n_product = 5
n_consumer = 100

# for simulation 

In [2]:
# input: market characteristics (x), and instrument 
def price_gen(x,instrument, noise_variance): #for a given x and instrument 
    noise = np.random.normal(0, noise_variance, n_product)
    price = np.random.normal(0.6,noise_variance) * instrument + np.random.normal(0.4,noise_variance) * x + noise
    return price
# output: engogenous price for each market set

# input: ALL parameters + product characteristics data 
def utility_gen(parameter_guess, sigma_guess, x, p): #return a matrix of utility 
    alpha, beta = parameter_guess[0], parameter_guess[1]
    sigma_alpha, sigma_beta = sigma_guess[0], sigma_guess[1] 
    utility_out = []
    shock_alpha = np.random.normal(0, 1, n_consumer)
    shock_beta = np.random.normal(0, 1, n_consumer)
    alpha_i = alpha + sigma_alpha * shock_alpha
    beta_i = beta + sigma_beta * shock_beta
    utility_out = np.outer(beta_i, x) - np.outer(alpha_i, p) #by formula 
    return utility_out 


# input: simulated utilities
def market_gen(utility_in, market_variance): #add market-specific shifter to each utility matrix
    market_core = np.random.uniform(-3,3)
    # one single shift central value but varies a little by person! 
    market_shifter = np.random.normal(market_core, market_variance, utility_in.shape)
    utility_out = utility_in + market_shifter
    return utility_out
# output: simulated AND market-shifted utilities

# input: utilities in each market 
def choice_prob(m_utilities): #for each market
    # print("initial utilities", m_utilities[:5])
    no_consumer, no_product = m_utilities.shape[0], m_utilities.shape[1]
    adjusted_utilities = np.clip(m_utilities, -100, 100) #clip to avoid overflow
    prob_choice = np.exp(adjusted_utilities)
    sum_utility = []
    for consumer in range(no_consumer):
        temp = 0 
        for product in range(no_product):
            temp += np.exp(m_utilities[product][consumer])
        sum_utility.append(temp)
    sum_utility = np.array(sum_utility) + 1
    prob_choice_out = prob_choice/sum_utility[:, None]
    return prob_choice_out
# output: choice prob matrix for each market 


# for estimation 

In [3]:
# input: a matrix of utilities for each consumer x product

# using utility_gen to generate utilities (given guess) for each consumer x product
def predicted_share(utilities_matrix, delta_guess): #for each market
    # print("initial", utilities_matrix[:5])
    # print("delta guess", delta_guess[:5])
    adjusted_utilities = utilities_matrix + delta_guess
    # print("adjusted", adjusted_utilities[:5])
    choice_prob_matrix = choice_prob(adjusted_utilities)
    # print("choice matrix", choice_prob_matrix)
    pred_share = choice_prob_matrix.sum(axis=0)/n_consumer
    pred_share = np.maximum(pred_share, 1e-10)  # Prevent division by zero
    return pred_share
# output: predicted share for each product in each market

# input: a matrix of utilities for each consumer x product + observed share 
def contraction_mapping(given_utility, observed_share, tolerance): #return converged mean utility 
    delta_guess = np.zeros(len(observed_share))
    difference = 1
    while difference > tolerance:
        adjusted_share = predicted_share(given_utility, delta_guess)
        safe_observed_share = np.clip(observed_share, 1e-10, None)  # Prevent division by zero
        delta_new = delta_guess + np.log(safe_observed_share) - np.log(adjusted_share)
        # print(safe_observed_share, safe_adjusted_share)
        difference = np.max(np.abs(delta_new - delta_guess))
        # print("delta guess", delta_guess)
        delta_guess = delta_new
    return delta_guess #return converged guess 
# output: converged mean utility for each product in each market

# input: market datase + all guesses 
def mean_utility(input_dataset, sigma_guess, parameter_guess, tolerance): #return mean utility for all 
    appending = []
    for i in range(n_market):
        market_i = input_dataset[input_dataset['market_id'] == i]
        # print("market_i printed", market_i)
        observed_share = market_i[market_i['market_id'] == i]['share'].values
        # simulate utilities given guess
        sim_utilities = pd.DataFrame(utility_gen(parameter_guess, sigma_guess, market_i['x'].values, market_i['price'].values))
        # print("simulated utilities", sim_utilities)
        appending.extend(contraction_mapping(sim_utilities, observed_share, tolerance))
        # print("done with market", i)
    # print("found appending", appending)
    return appending 
# output: an array of mean utility to appending into input dataset

def estimate_parameter(input_dataset): #return estimated alpha and beta parameters 
    X = input_dataset[['x','price']].values 
    Z = input_dataset[['instrument','price']].values
    y = input_dataset['mean_utility'].values
    # First stage
    Pi = inv(Z.T @ Z) @ (Z.T @ X)
    X_hat = Z @ Pi #generate predicted X ==
    # Second stage
    theta = inv(X_hat.T @ X_hat) @ (X_hat.T @ y)
    alpha_hat, beta_hat = -theta[1], theta[0]
    return alpha_hat, beta_hat 
